## Funzionamento pratico del programma

### 1. Setup iniziale
- Il programma parte dal file `main.py`, che rappresenta il punto di ingresso principale del sistema.
- All'avvio, `main.py` carica:
  - i **parametri generali** dal file `data/config.yaml` (es. indirizzo IP del robot, parametri di sicurezza, posizioni di default, soglie per la visione);
  - la **lista delle parole disponibili** da `data/words.json`;
  - eventuali **dati di calibrazione** (camera, marker ArUco, coordinate del tavolo) dalla cartella `data/calibration/`.


### 2. Selezione e preparazione della parola

Il modulo `game_logic.py` ha la funzione `select_word()` che sceglie una parola casuale dal dataset (o in base al livello impostato).

La parola scelta viene comunicata al giocatore tramite TTS. A questo punto il robot posiziona sul tavolo alcune lettere iniziali, lasciando all’utente il compito di completarle.

NB: All’interno di una funzione:
- Recupera le coordinate della griglia dal file di configurazione.
- Fa muovere il robot in sequenza: sopra la cella → scende → apre o chiude il gripper → torna in posizione di sicurezza.

### 3. Interazione con l’utente

Dopo che il robot ha posato i blocchi, il sistema invita l’utente a completare la parola.

Il programma aspetta poi un input manuale o vocale

### 4. Riconoscimento visivo

Quando l’utente segnala di aver finito, `main.py` richiama `detect_letters()` di `vision.py`.

Questa funzione:
- Cattura un frame dalla camera
- Corregge la prospettiva in base ai marker ArUco
- Segmenta i blocchi
- Riconosce le lettere tramite OCR (pytesseract) o classificatore
- Restituisce una lista strutturata, ad esempio: [{'char': 'G', 'x': 120, 'y': 340}, {'char': 'A', 'x': 200, 'y': 340}, ...]

### 5. Verifica della parola e logica del gioco

Il risultato del riconoscimento viene passato a `game_logic.py`, che confronta la sequenza delle lettere con la parola target. Se la parola corrisponde, `check_word()` restituisce True, altrimenti False.

### 6. Feedback e movimenti del robot

Se `success == True`:
say("Perfetto! Hai completato la parola!")
robot.celebrate()
La funzione `celebrate()` magari esegue una piccola coreografia col braccio robotico (es. movimento su-giù o oscillazione rapida).

Se `success == False`:
say("Ops! Alcune lettere non sono corrette, prova di nuovo!")
robot.encourage()
`encourage()` può essere, ad esempio, una piccola rotazione del polso o accenno laterale.

### 7. Log dei risultati e salvataggio

A ogni sessione è associato un salvataggio tramite funzione `log_result()` in `utils.py`:
from utils import log_result
log_result(target_word, detected_letters, success)

text
Genera un file CSV o JSON in `data/test_logs/` con, per ogni sessione:
timestamp, parola, lettere_rilevate, esito, tempo_totale



---

## Note Personali

- Fissare con scotch/qualcosa di rigido le postazioni dei blocchi fissi per semplificare movimenti robot e visione artificiale (così devo solo capire che lettera è e non sov'è il blocco anche)

- fissare delle portazioni del robot muovendolo a mano e così tenere conto delle posizioni dell parole

- separiamo vocali e consonanti (di vocali 2 copie mentre di consonanti magari giusto un paio di doppie)

- le parole più lunghe di 6 lettere non ci sono, dovremo dire che lo faranno nelle ricerche future volendo insieme anche a tipo frasi o altri tipi di interazioni/feedback

- se facciamo i log possiamo proporre una possibile continuazione del progetto come una profilazione degli studenti che tipo monitora il loro miglioramente in cui è anche ointegrata un AI/ un sistema che è in grado di analizzare gli errori e capire le carenze della gente per consigliare le parole più adatte